In [4]:
#Converter from NL to Protocol
import os
import sys
import numpy as np
import time
import MeCab
import codecs
import json
#
actionS = ['estimate','comingout','divined','vote','None']
agentS = ['Agent[01]','Agent[02]','Agent[03]','Agent[04]','Agent[05]','None']
roleS = ['villager','seer','wolf','possesed','None']
specieS = ['human','wolf','None']
ACTION_NUM = len(actionS)#estimate,divine,co,vote
AGENT_NUM =len(agentS)#15
ROLE_NUM = len(roleS)#
SPECIES_NUM = len(specieS)
maxRemarkNum = 20
maxWordNum = 100
vocabNum = 1000#one-hotベクトルサイズ  
#(発言数,プロトコル)->(発言数,単語数,語彙数)
#(プロトコル)->(単語数,語彙数)
class Parser:
    """ MeCab を使用して文章を分割する """
    #listFlag = True#２次元リストで返すか(True)、１次元リストで返すか(False)
    modeS = [1]#
    tagger = None

    def __init__(self,aModeS=None):# 第一引数は自インスタンス(self)
        self.modeS = aModeS
        self.tagger = MeCab.Tagger('')
        #self.tagger = MeCab.Tagger('-d ../MeCab-Dict/')
        parsed0 = self.tagger.parse('')#これ重要！！！！

    def getParse(self,aText):#表層文字だけを分かち書きにして返す
        #parsed0 = self.tagger.parse('')#これ重要！！！！
        #node = self.tagger.parseToNode(aText)
        #m = self.tagger("-Owakati")
        m = MeCab.Tagger("-Owakati")
        mecab_string = m.parse(aText)
        mecab_list = []
        for t in m.parse(aText).split():
            mecab_list.append(t)
        return mecab_list, mecab_string


def prot2OneHot(aProtocol):
    actionVec = np.zeros(ACTION_NUM)
    action = aProtocol[0]
    actionVec[actionS.index(action)]=1
    #
    subjectVec = np.zeros(AGENT_NUM)
    subject = aProtocol[1]
    subjectVec[agentS.index(subject)]=1
    #
    targetVec = np.zeros(AGENT_NUM)
    target = aProtocol[2]
    targetVec[agentS.index(target)]=1
    #
    roleVec = np.zeros(ROLE_NUM)
    role = aProtocol[3]
    roleVec[roleS.index(role)]=1
    #
    speciesVec = np.zeros(SPECIES_NUM)
    species = aProtocol[4]
    speciesVec[specieS.index(species)]=1
    #
    #protVec = np.concatenate([actionVec,subjectVec,targetVec,roleVec,speciesVec])
    protVec = [actionVec,subjectVec,targetVec,roleVec,speciesVec]
    protVec = [np.array(actionVec),np.array(subjectVec),np.array(targetVec),np.array(roleVec),np.array(speciesVec)]
    return(protVec)

def protS2OneHot(aProtocolList):
    actionVecS = []
    subjectVecS = []
    targetVecS = []
    roleVecS = []
    speciesVecS = []
    for protocol in aProtocolList:
        protVec = prot2OneHot(protocol)
        actionVecS.append(protVec[0])
        subjectVecS.append(protVec[1])
        targetVecS.append(protVec[2])
        roleVecS.append(protVec[3])
        speciesVecS.append(protVec[4])
        #protVecS.append(protVec)
    #actionArray = np.concatenate(actionVecS)
    #subjectArray = np.concatenate(subjectVecS)
    #targetArray = np.concatenate(targetVecS)
    #roleArray = np.concatenate(roleVecS)
    #speciesArray = np.concatenate(speciesVecS)
    actionArray = np.array(actionVecS)
    subjectArray = np.array(subjectVecS)
    targetArray = np.array(targetVecS)
    roleArray = np.array(roleVecS)
    speciesArray = np.array(speciesVecS)
    return(actionArray,subjectArray,targetArray,roleArray,speciesArray)

mecab_parser = Parser()
#chat_train_txt = 'chat_train.txt'
#chat_train_prot = 'chat_prot.txt'
divine_train_txt = 'divine_train2.txt'
divine_train_prot = 'divine_prot2.txt'
co_train_txt = 'co_train.txt'
co_train_prot = 'co_prot.txt'
vote_train_txt = 'vote_train.txt'
vote_train_prot = 'vote_prot.txt'
def txt2OneHot(aWordList,aVocab):
    vocabNum = len(aVocab)
    txtLen = len(aWordList)
    oneHotS = []
    vocab = list(aVocab)
    for word in aWordList:
        oneHotVec = np.zeros(vocabNum)
        index = vocab.index(word)
        oneHotVec[index] = 1
        oneHotS.append(oneHotVec)
    #oneHotArray = np.array(oneHotS)
    return(oneHotS)

def sent2OneHot(aSentList,aMaxWordNum,aVocab):
    sentOneHot = []
    for txt in aSentList:
        oneHot = txt2OneHot(txt,aVocab)
        while (len(oneHot) < maxWordNum):
            vec = np.zeros(len(aVocab))
            #print(np.shape(oneHot))
            #print(np.shape(vec))
            #oneHot = np.concatenate([oneHot,vec])
            oneHot.append(vec)
        sentOneHot.append(oneHot)
    oneHotArray = np.array(sentOneHot)
    return(oneHotArray) 

def makeDataTxt(aFileName):
    textS = []
    vocab_list = []
    maxWordNum = 0
    for line in codecs.open('./train3/'+aFileName, 'r', 'utf-8'):
        parsed_list,parsed = mecab_parser.getParse(line)
        #print(parsed)
        vocab_list.extend(parsed_list)
        textS.append(parsed_list)
        maxWordNum = max(maxWordNum,len(parsed_list))
    vocab = set(vocab_list)
    return(textS,maxWordNum,vocab)

def makeDataProt(aFileName):
    protS = []
    for line in codecs.open('./train3/'+aFileName, 'r', 'utf-8'):
        protocol = line.split()
        protS.append(protocol)
    return(protS)

#chat_textS,maxWordNum,vocab= makeDataTxt(chat_train_txt)
#
divine_textS,maxWordNum2,vocab= makeDataTxt(divine_train_txt)
#vocabList = list(vocab)
#vocabList2 = list(vocab2)
#vocabList.extend(vocabList2)
#vocab = set(vocabList)
#maxWordNum = max(maxWordNum,maxWordNum2)
#
co_textS,maxWordNum2,vocab2= makeDataTxt(co_train_txt)
vocabList = list(vocab)
vocabList2 = list(vocab2)
vocabList.extend(vocabList2)
vocab = set(vocabList)
maxWordNum = max(maxWordNum,maxWordNum2)
#
vote_textS,maxWordNum2,vocab2= makeDataTxt(vote_train_txt)
vocabList = list(vocab)
vocabList2 = list(vocab2)
vocabList.extend(vocabList2)
vocab = set(vocabList)
maxWordNum = max(maxWordNum,maxWordNum2)
#
#chatTrain_txt = sent2OneHot(chat_textS,maxWordNum,vocab)
divineTrain_txt = sent2OneHot(divine_textS,maxWordNum,vocab)
coTrain_txt = sent2OneHot(co_textS,maxWordNum,vocab)
voteTrain_txt = sent2OneHot(vote_textS,maxWordNum,vocab)
#train_txt = np.concatenate([chatTrain_txt,divineTrain_txt,coTrain_txt,voteTrain_txt])
train_txt = np.concatenate([divineTrain_txt,coTrain_txt,voteTrain_txt])
#
#train_protS = makeDataProt(chat_train_prot)
train_protS = makeDataProt(divine_train_prot)
#train_protS.extend(divine_protS)
co_protS = makeDataProt(co_train_prot)
train_protS.extend(co_protS)
vote_protS = makeDataProt(vote_train_prot)
train_protS.extend(vote_protS)
actionProt,subjectProt,targetProt,roleProt,speciesProt = protS2OneHot(train_protS)
#
vocab_fileName = 'vocab.txt'
def writeVocab():
    #f = open(vocab_fileName, 'w')
    f = codecs.open(vocab_fileName, 'w', 'utf-8')
    for word in vocab:
        f.write(word)
    f.close()
    return
writeVocab()
vocabNum = len(vocab)
#==========================================
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, Reshape, Embedding, Flatten, Dropout, RepeatVector
from keras.layers import Concatenate
from keras.layers import LSTM, Bidirectional
from keras.optimizers import RMSprop
from keras.utils import multi_gpu_model
losseS = ['mean_squared_error', 'categorical_crossentropy']
loss = losseS[0]
def makeP2TModel():
    actionInputs = Input(shape=(ACTION_NUM,))
    subjectInputs = Input(shape=(AGENT_NUM,))
    targetInputs = Input(shape=(AGENT_NUM,))
    roleInputs = Input(shape=(ROLE_NUM,))
    speciesInputs = Input(shape=(SPECIES_NUM,))
    #verbInputs = Input(shape=(verb,))
    #stateInputs = Input()
    protocolInputs = Concatenate()([actionInputs,subjectInputs,targetInputs,roleInputs,speciesInputs])
    #dayInputs = Input(1,)
    #propertyInputs = Concatenate()([dayInputs])
    repeatInputs = RepeatVector(maxWordNum)(protocolInputs)
    lstm1 = LSTM(100,return_sequences=True)(repeatInputs)
    lstm2 = LSTM(50,return_sequences=True)(lstm1)
    remarkOut = Dense(vocabNum)(lstm2)
    model = Model(inputs=[actionInputs, subjectInputs,targetInputs,roleInputs,speciesInputs], outputs=[remarkOut])
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return(model)
modelP2T = makeP2TModel()
batch_size = 5
epochs=5
#print(len(chatTrain_prot))
modelP2T.fit([actionProt,subjectProt,targetProt,roleProt,speciesProt],train_txt,batch_size=batch_size, epochs=epochs, verbose=1)
json_string = modelP2T.to_json()
fP2T = open('weightP2T.json', 'w')
json.dump(json_string, fP2T)
modelP2T.save_weights('paramP2T.hdf5')
#print(modelP2T.predict([actionProt,subjectProt,targetProt,roleProt,speciesProt]))
#model.load_weights('param.hdf5')
def makeT2PModel():
    remarkInput = Input(shape=(maxWordNum,vocabNum))
    lstm1 = LSTM(100)(remarkInput)
    actionOut = Dense(ACTION_NUM)(lstm1)
    subjectOut = Dense(AGENT_NUM)(lstm1)
    targetOut = Dense(AGENT_NUM)(lstm1)
    roleOut = Dense(ROLE_NUM)(lstm1)
    speciesOut = Dense(SPECIES_NUM)(lstm1)
    model = Model(inputs=[remarkInput], outputs=[actionOut, subjectOut,targetOut,roleOut,speciesOut])
    model.compile('adam', 'mean_squared_error', metrics=['accuracy'])
    model.summary()
    return(model)
modelT2P = makeT2PModel()
modelT2P.fit(train_txt,[actionProt,subjectProt,targetProt,roleProt,speciesProt],batch_size=batch_size, epochs=epochs, verbose=1)
json_string = modelT2P.to_json()
fT2P = open('weightT2P.json', 'w')
json.dump(json_string, fT2P)
modelT2P.save_weights('paramT2P.hdf5')
#選択候補となる配列から値を取り出す
def sample(aPreds,aTemperature=1.0):
    #print(aPreds)
    for i,val in enumerate(aPreds):
        if(val<0):
            aPreds[i] = 0
        elif(val>1.0):
            aPreds[i] = 1.0
            #return(i)
    aPreds = np.asarray(aPreds).astype('float64')#aPreds は0.0~1.0の値を取る実数配列
    #aPreds = np.log(aPreds)/aTemperature
    exp_preds = np.exp(aPreds)
    #aPreds = exp_preds/np.sum(exp_preds)#aPredsの値の合計が１になるように正規化を行う
    aPreds = aPreds/np.sum(aPreds)
    probas = np.random.multinomial(1,aPreds,1)#multinomial 関数によって aPreds 内の値に応じた確率で試行を行って one-hot ベクトルにするs
    return(np.argmax(probas))#numpy.argmax() で最大値要素の内で最も小さいインデックスを返す つまり、１である要素の添字を返す
#
def mat2Index(aMatrix):
    indexS = []
    for vec in aMatrix:
        indexS.append(sample(vec))
    return(indexS)

#pred = modelT2P.predict(train_txt[-1])
#actionPred = (pred[0])
pred = modelT2P.predict(train_txt[-1])
indexS = mat2Index(pred)
uttr = ''
for index in indexS:
    uttr+=vocab[index]
print(uttr)

/home/itolab/virtualenvs/yoshino-env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
input_5 (I

ValueError: Error when checking : expected input_6 to have 3 dimensions, but got array with shape (100, 44)

In [7]:
def sample(aPreds,aTemperature=1.0):
    #print(aPreds)
    #aPreds = aPreds[0]
    for i,val in enumerate(aPreds):
        if(val<0):
            aPreds[i] = 0
        elif(val>1.0):
            aPreds[i] = 1.0
            #return(i)
    aPreds = np.asarray(aPreds).astype('float64')#aPreds は0.0~1.0の値を取る実数配列
    #aPreds = np.log(aPreds)/aTemperature
    exp_preds = np.exp(aPreds)
    aPreds = exp_preds/np.sum(exp_preds)#aPredsの値の合計が１になるように正規化を行う
    aPreds = aPreds/np.sum(aPreds)
    probas = np.random.multinomial(1,aPreds,1)#multinomial 関数によって aPreds 内の値に応じた確率で試行を行って one-hot ベクトルにするs
    return(np.argmax(probas))#numpy.argmax() で最大値要素の内で最も小さいインデックスを返す つまり、１である要素の添字を返す
#
def mat2Index(aMatrix):
    indexS = []
    for vec in aMatrix:
        indexS.append(sample(vec))
    return(indexS)
pred = modelP2T.predict([actionProt[200:201],subjectProt[200:201],targetProt[200:201],roleProt[200:201],speciesProt[200:201]])
print([actionProt[200:201],subjectProt[200:201],targetProt[200:201],roleProt[200:201],speciesProt[200:201]])
print(np.shape(pred))
indexS = mat2Index(pred[0])
uttr = ''
vocabList = list(vocab)
for index in indexS:
    uttr+=vocabList[index]
print(uttr)

[array([[0., 0., 1., 0., 0.]]), array([[0., 0., 0., 0., 0., 1.]]), array([[0., 1., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 1.]]), array([[0., 1., 0.]])]
(1, 100, 44)
に占っなたら05ましよです投票のよです霊媒師の。人間投票占っ01COAgent占っ結果ようた占いよ霊媒師COや霊媒師村人だ結果05こそにん人狼結果[私こそ05て裏切り者霊媒師んますよ占っ結果だっはだはをた霊媒師です02にによ人狼02しは[の]こそだっです裏切り者は]先。はんだますだよは村人裏切り者こそにも03人狼03人間05ところだ先]
